In [0]:
from tensorflow.keras.layers import Dense, Flatten, Dropout, Convolution2D, MaxPooling2D
from tensorflow.keras.preprocessing.image import load_img, img_to_array, ImageDataGenerator
from tensorflow.keras.models import Sequential
import numpy as np
from tensorflow.keras import utils
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, EarlyStopping

**Load our data**

In [0]:
train = np.loadtxt('train.csv',skiprows=1,delimiter=',')

In [0]:
test = np.loadtxt('test.csv',skiprows=1,delimiter=',')

**Let's extract target (y) and features and preprocess them by deleting by 255 and reshaping X, converting target to categorical**

In [0]:
X = train[:,1:]
X = X.reshape(X.shape[0],28,28,1)
X /= 255.0
y = train[:,0]
y = utils.to_categorical(y,10)
input_shape=(28,28,1)

Split our data

In [0]:
X_tr, X_ts, y_tr, y_ts = train_test_split(X,y,random_state=42)

**Use ImageDataGenerator to expand our data**

In [0]:
datagen = ImageDataGenerator(zoom_range=0.2,height_shift_range=0.15,width_shift_range=0.15,
                             rotation_range=0.2)

**To construct a model we will use Keras-tuner**

In [7]:
pip install keras-tuner

     |████████████████████████████████| 61kB 4.6MB/s 
  Created wheel for keras-tuner: filename=keras_tuner-1.0.1-cp36-none-any.whl size=73200 sha256=e3116eb3534d9f17042509876ba82aa33f1878f5bd6e7df9cd760bcb8a21496a
  Stored in directory: /root/.cache/pip/wheels/b9/cc/62/52716b70dd90f3db12519233c3a93a5360bc672da1a10ded43
  Created wheel for terminaltables: filename=terminaltables-3.1.0-cp36-none-any.whl size=15356 sha256=568618af5c5ebc702f5c0b57db147ef026231dbfad52e1b679a23dbf37ad865d
  Stored in directory: /root/.cache/pip/wheels/30/6b/50/6c75775b681fb36cdfac7f19799888ef9d8813aff9e379663e
Successfully built keras-tuner terminaltables


Let's use simple RandomSearch. We will loking for an **appropriate activation function, optimizer, number of units in Dense layer, proportion of Dropout and number of filters** in Conv2d layers. I've chosen 3 Conv2d layers in our model.


In [0]:
from kerastuner.tuners import RandomSearch

In [0]:
def build_model(hp):
  model = Sequential()
  activation_choice = hp.Choice('activation',['relu','elu'])
  optimizer = hp.Choice('optimizer',['SGD','adam','rmsprop'])
  model.add(Convolution2D(filters=hp.Int('filters',min_value = 32,max_value = 512,step=32),
                            kernel_size=(5,5),padding='same',activation=activation_choice,
                          input_shape=input_shape))
  model.add(Convolution2D(filters=hp.Int('filters',min_value = 32,max_value = 512,step=32),
                            kernel_size=(5,5),padding='same',activation=activation_choice))
  model.add(MaxPooling2D())
  model.add(Dropout(hp.Float('dropout1', 0, 0.5, step=0.1)))

  model.add(Convolution2D(filters=hp.Int('filters2',min_value = 32,max_value = 512,step=32),
                            kernel_size=(3,3),padding='same',activation=activation_choice))
  model.add(Convolution2D(filters=hp.Int('filters2',min_value = 32,max_value = 512,step=32),
                            kernel_size=(3,3),padding='same',activation=activation_choice))
  model.add(MaxPooling2D())
  model.add(Dropout(hp.Float('dropout2', 0, 0.5, step=0.1)))

  model.add(Convolution2D(filters=hp.Int('filters3',min_value = 32,max_value = 512,step=32),
                            kernel_size=(3,3),padding='same',activation=activation_choice))
  model.add(Convolution2D(filters=hp.Int('filters3',min_value = 32,max_value = 512,step=32),
                            kernel_size=(3,3),padding='same',activation=activation_choice))
  model.add(MaxPooling2D())
  model.add(Dropout(hp.Float('dropout3', 0, 0.5, step=0.1)))
  
  model.add(Flatten())
  model.add(Dense(units = hp.Int('units',64,512,step=32),activation=activation_choice))
  model.add(Dropout(hp.Float('dropout4', 0, 0.5, step=0.1)))
  model.add(Dense(10,activation='softmax'))
  model.compile(metrics=['accuracy'],loss='categorical_crossentropy',optimizer=optimizer)
  return model

Creating a tuner


In [0]:
tuner = RandomSearch(build_model,'val_accuracy',seed=42, max_trials=5)

Let's look at the summary of our tuners

In [15]:
tuner.search_space_summary()

And we are ready to start a search for the best hyperparameters, remember not only use your validation data, but also include ImageDataGenerator for your train data. I haven't chage the batch size of 32 (default) and I've included EarlyStopping to prevent overfitiing.

In [16]:
tuner.search(datagen.flow(X_tr, y_tr), steps_per_epoch=X_tr.shape[0]//32,
             validation_data=(X_ts, y_ts),epochs=10,
             callbacks=[EarlyStopping(patience=2)])

Epoch 1/10
984/984 [==============================] - 25s 26ms/step - loss: 1.8481 - accuracy: 0.3619 - val_loss: 0.3627 - val_accuracy: 0.8972
Epoch 2/10
984/984 [==============================] - 25s 25ms/step - loss: 0.3641 - accuracy: 0.8885 - val_loss: 0.1290 - val_accuracy: 0.9610
Epoch 3/10
984/984 [==============================] - 25s 25ms/step - loss: 0.2000 - accuracy: 0.9384 - val_loss: 0.0932 - val_accuracy: 0.9727
Epoch 4/10
984/984 [==============================] - 25s 25ms/step - loss: 0.1505 - accuracy: 0.9554 - val_loss: 0.0737 - val_accuracy: 0.9770
Epoch 5/10
984/984 [==============================] - 25s 25ms/step - loss: 0.1235 - accuracy: 0.9626 - val_loss: 0.0618 - val_accuracy: 0.9815
Epoch 6/10
984/984 [==============================] - 25s 25ms/step - loss: 0.1087 - accuracy: 0.9667 - val_loss: 0.0631 - val_accuracy: 0.9792
Epoch 7/10
984/984 [==============================] - 25s 25ms/step - loss: 0.0930 - accuracy: 0.9707 - val_loss: 0.0461 - val_accuracy:

Epoch 1/10
984/984 [==============================] - 55s 56ms/step - loss: 21.2050 - accuracy: 0.1009 - val_loss: 2.3219 - val_accuracy: 0.0985
Epoch 2/10
984/984 [==============================] - 54s 55ms/step - loss: 2.4192 - accuracy: 0.1015 - val_loss: 2.3219 - val_accuracy: 0.1091
Epoch 3/10
984/984 [==============================] - 54s 55ms/step - loss: 2.3750 - accuracy: 0.1028 - val_loss: 2.3339 - val_accuracy: 0.1081
Epoch 4/10
984/984 [==============================] - 54s 55ms/step - loss: 3.2562 - accuracy: 0.1016 - val_loss: 2.3205 - val_accuracy: 0.0976
Epoch 5/10
984/984 [==============================] - 54s 55ms/step - loss: 2.9350 - accuracy: 0.1038 - val_loss: 2.3166 - val_accuracy: 0.0975
Epoch 6/10
984/984 [==============================] - 54s 55ms/step - loss: 2.3371 - accuracy: 0.0987 - val_loss: 2.3203 - val_accuracy: 0.0975
Epoch 7/10
984/984 [==============================] - 54s 55ms/step - loss: 2.3336 - accuracy: 0.1040 - val_loss: 2.3187 - val_accuracy

Epoch 1/10
984/984 [==============================] - 20s 20ms/step - loss: 0.4173 - accuracy: 0.8593 - val_loss: 0.0541 - val_accuracy: 0.9839
Epoch 2/10
984/984 [==============================] - 20s 20ms/step - loss: 0.1040 - accuracy: 0.9699 - val_loss: 0.0535 - val_accuracy: 0.9840
Epoch 3/10
984/984 [==============================] - 20s 20ms/step - loss: 0.0826 - accuracy: 0.9758 - val_loss: 0.0306 - val_accuracy: 0.9905
Epoch 4/10
984/984 [==============================] - 20s 20ms/step - loss: 0.0682 - accuracy: 0.9805 - val_loss: 0.0357 - val_accuracy: 0.9896
Epoch 5/10
984/984 [==============================] - 20s 20ms/step - loss: 0.0658 - accuracy: 0.9813 - val_loss: 0.0309 - val_accuracy: 0.9927


Epoch 1/10
984/984 [==============================] - 42s 42ms/step - loss: 1.0318 - accuracy: 0.6248 - val_loss: 0.0872 - val_accuracy: 0.9761
Epoch 2/10
984/984 [==============================] - 41s 42ms/step - loss: 0.1340 - accuracy: 0.9634 - val_loss: 0.0491 - val_accuracy: 0.9858
Epoch 3/10
984/984 [==============================] - 41s 42ms/step - loss: 0.0959 - accuracy: 0.9719 - val_loss: 0.0483 - val_accuracy: 0.9867
Epoch 4/10
984/984 [==============================] - 41s 42ms/step - loss: 0.0820 - accuracy: 0.9772 - val_loss: 0.0342 - val_accuracy: 0.9898
Epoch 5/10
984/984 [==============================] - 41s 42ms/step - loss: 0.0764 - accuracy: 0.9794 - val_loss: 0.0444 - val_accuracy: 0.9872
Epoch 6/10
984/984 [==============================] - 41s 42ms/step - loss: 0.0696 - accuracy: 0.9814 - val_loss: 0.0355 - val_accuracy: 0.9892


Epoch 1/10
984/984 [==============================] - 38s 39ms/step - loss: 0.3981 - accuracy: 0.8721 - val_loss: 0.0872 - val_accuracy: 0.9765
Epoch 2/10
984/984 [==============================] - 37s 38ms/step - loss: 0.0991 - accuracy: 0.9703 - val_loss: 0.0850 - val_accuracy: 0.9754
Epoch 3/10
984/984 [==============================] - 38s 38ms/step - loss: 0.0750 - accuracy: 0.9780 - val_loss: 0.0500 - val_accuracy: 0.9839
Epoch 4/10
984/984 [==============================] - 37s 38ms/step - loss: 0.0644 - accuracy: 0.9820 - val_loss: 0.0477 - val_accuracy: 0.9866
Epoch 5/10
984/984 [==============================] - 38s 38ms/step - loss: 0.0590 - accuracy: 0.9824 - val_loss: 0.0367 - val_accuracy: 0.9903
Epoch 6/10
984/984 [==============================] - 38s 38ms/step - loss: 0.0537 - accuracy: 0.9844 - val_loss: 0.0337 - val_accuracy: 0.9910
Epoch 7/10
984/984 [==============================] - 37s 38ms/step - loss: 0.0484 - accuracy: 0.9862 - val_loss: 0.0335 - val_accuracy:

INFO:tensorflow:Oracle triggered exit


So as we see our best accurecy score for validation data is 99.27%

In [17]:
best_models = tuner.get_best_models(2)
for model in best_models:
  model.evaluate(X_ts,y_ts)
  print()

329/329 [==============================] - 2s 6ms/step - loss: 0.0309 - accuracy: 0.9927

329/329 [==============================] - 4s 11ms/step - loss: 0.0337 - accuracy: 0.9910



That's why we will reinstantiate our model using best chosen hyperparameters

In [0]:
best_hp = tuner.get_best_hyperparameters()[0]
tuned_model = tuner.hypermodel.build(best_hp)

In [37]:
tuned_model.fit(datagen.flow(X_tr, y_tr), steps_per_epoch=X_tr.shape[0]//32,
             validation_data=(X_ts, y_ts),epochs=8,
             callbacks=[EarlyStopping(patience=2)])

Epoch 1/8
984/984 [==============================] - 20s 20ms/step - loss: 0.0396 - accuracy: 0.9895 - val_loss: 0.0410 - val_accuracy: 0.9919
Epoch 2/8
984/984 [==============================] - 20s 20ms/step - loss: 0.0390 - accuracy: 0.9892 - val_loss: 0.0369 - val_accuracy: 0.9913
Epoch 3/8
984/984 [==============================] - 20s 20ms/step - loss: 0.0395 - accuracy: 0.9888 - val_loss: 0.0318 - val_accuracy: 0.9924
Epoch 4/8
984/984 [==============================] - 20s 20ms/step - loss: 0.0413 - accuracy: 0.9893 - val_loss: 0.0292 - val_accuracy: 0.9933
Epoch 5/8
984/984 [==============================] - 20s 20ms/step - loss: 0.0363 - accuracy: 0.9906 - val_loss: 0.0401 - val_accuracy: 0.9910
Epoch 6/8
984/984 [==============================] - 20s 20ms/step - loss: 0.0424 - accuracy: 0.9883 - val_loss: 0.0365 - val_accuracy: 0.9926


In [38]:
tuned_model.evaluate(X_ts,y_ts)

329/329 [==============================] - 2s 6ms/step - loss: 0.0365 - accuracy: 0.9926


[0.03653445467352867, 0.9925714135169983]

A little bit preprocess test data

In [0]:
x_test = test.reshape(test.shape[0],28,28,1)
x_test /= 255.0

Let's predict with our test data

In [0]:
predictions = tuned_model.predict(x_test)

In [40]:
predictions = np.argmax(predictions,axis=1)
predictions[:10]

array([2, 0, 9, 0, 3, 7, 0, 3, 0, 3])

**Save our file in an appropriate form for the Kaggle. The score is 99.357%. That's a nice job!**

In [0]:
submission = np.column_stack((range(1,predictions.shape[0]+1),predictions))
np.savetxt('submission.csv',submission, header='ImageId,Label', fmt='%d,%d',comments='')

In [25]:
!head submission.csv

ImageId,Label
1,2
2,0
3,9
4,0
5,3
6,7
7,0
8,3
9,0
